# Build Model for Training, and Test accuracy

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from PIL import Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
import cPickle as pickle
import tensorflow as tf

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline
np.set_printoptions(threshold=np.inf)

/usr/local/lib/python2.7/dist-packages/ipykernel/pylab/config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""
/usr/local/lib/python2.7/dist-packages/ipykernel/pylab/config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
/usr/local/lib/python2.7/dist-packages/ipykernel/pylab/config.py:85: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  use `figure_formats` instead)""")
/usr/local/lib/python2.7/dist-packages/ipykernel/pylab/config.py:95: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag()

### Deserialize

In [2]:
train_pickle_file = 'SVHN_basic_train.pickle'

with open(train_pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_labels = save['train_image_labels']
  train_dataset = save['train_dataset']
  del save  # hint to help gc free up memory
  print('Training set', train_labels.shape, train_dataset.shape)
print(train_labels[0:2,:])

Training set (33402, 6) (33402, 64, 64)
[[ 2.  0.  0.  0.  1.  9.]
 [ 2.  0.  0.  0.  2.  3.]]


In [3]:
test_pickle_file = 'SVHN_basic_test.pickle'

with open(test_pickle_file, 'rb') as f:
  save = pickle.load(f)
  test_labels = save['test_image_labels']
  test_dataset = save['test_dataset']
  del save  # hint to help gc free up memory
  print('Test set', test_labels.shape, test_dataset.shape)

Test set (13068, 6) (13068, 64, 64)


####TODO: Randomize and Shuffle
####TODO: Use Boosting
####TODO: Use Simulated Annealing / Decay

###Reformat

In [4]:
### Reformat dataset to 1 channel
image_size=64
num_channels=1 #greyscale

def reformat(dataset):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels))\
        .astype(np.float32)
  return dataset

train_dataset = reformat(train_dataset)
test_dataset = reformat(test_dataset)
print("Reformatted shapes of datasets\n")
print("train_dataset.shape:",train_dataset.shape,", train_labels.shape:",train_labels.shape)
print("test_dataset.shape:", test_dataset.shape,", test_labels.shape:",test_labels.shape)

Reformatted shapes of datasets

train_dataset.shape: (33402, 64, 64, 1) , train_labels.shape: (33402, 6)
test_dataset.shape: (13068, 64, 64, 1) , test_labels.shape: (13068, 6)


#### Reformat dataset to 1 channel
image_size=64
num_channels=1 #greyscale

def reformat(dataset):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels))\
        .astype(np.float32)
  return dataset

train_dataset = reformat(train_dataset)
#test_dataset = reformat(test_dataset)
print("Reformatted shapes of datasets\n")
print("train_dataset.shape:",train_dataset.shape,", train_labels.shape:",train_labels.shape)
#print("test_dataset.shape:", test_dataset.shape,", test_labels.shape:",test_labels.shape)

### Define Accuracy

In [5]:
def accuracy(predictions, labels):
    label_count=labels.shape[0]
    #predictions=predictions.astype(int)
    print("predictions:\n",predictions)
    print("labels:\n", labels)
    len_preds=np.argmax(predictions[0,:,:],axis=1)
    digit_1_preds=np.argmax(predictions[1,:,:],axis=1)
    digit_2_preds=np.argmax(predictions[2,:,:],axis=1)
    digit_3_preds=np.argmax(predictions[3,:,:],axis=1)
    digit_4_preds=np.argmax(predictions[4,:,:],axis=1)
    digit_5_preds=np.argmax(predictions[5,:,:],axis=1)
    
    print("len preds:", len_preds)
    print(" digit_1_preds",digit_1_preds)
    print(" digit_2_preds",digit_2_preds)
    print(" digit_3_preds",digit_3_preds)
    print(" digit_4_preds",digit_4_preds)
    print(" digit_5_preds",digit_5_preds)
    
    len_accuracy=len_preds==labels[:,0]
    print("len_accuracy:",len_accuracy.shape)
    digit_1_accuracy=digit_1_preds==labels[:,1]
    print("digit_1_accuracy:",digit_1_accuracy.shape, digit_1_accuracy)
    digit_2_accuracy=digit_2_preds==labels[:,2]
    print("digit_2_accuracy:",digit_2_accuracy.shape, digit_2_accuracy)
    digit_3_accuracy=digit_3_preds==labels[:,3]
    print("digit_3_accuracy:",digit_3_accuracy.shape, digit_3_accuracy)
    digit_4_accuracy=digit_4_preds==labels[:,4]
    print("digit_4_accuracy:",digit_4_accuracy.shape, digit_4_accuracy)
    digit_5_accuracy=digit_1_preds==labels[:,5]
    print("digit_5_accuracy:",digit_5_accuracy.shape, digit_5_accuracy)
    
    
    complete_accuracy=np.concatenate((len_accuracy.reshape(1,label_count),digit_1_accuracy.reshape(1,label_count),\
                                      digit_2_accuracy.reshape(1,label_count),digit_3_accuracy.reshape(1,label_count), \
                                      digit_4_accuracy.reshape(1,label_count),digit_5_accuracy.reshape(1,label_count)), axis=0).T
    print("complete_accuracy:\n",complete_accuracy.shape, complete_accuracy)
    return 100.0 * (np.sum([np.all(row) for row in complete_accuracy])) / len(labels)
    #return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
    #      / predictions.shape[0])

#### Create Model

In [10]:
batch_size = 32
patch_size = 5
depth = 64
num_hidden = 1024
num_labels = 10 # 10 for 0-9
len_labels=6 #5 for lenghts0-5, and more than 5


graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, len_labels))

  tf_digit_masks_1= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_digit_masks_2= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_digit_masks_3= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_digit_masks_4= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_digit_masks_5= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  cnv_lyr1_weights = tf.get_variable("W1", shape=[patch_size, patch_size, num_channels, depth], \
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
  cnv_lyr1_biases = tf.Variable(tf.constant(0.01, shape=[depth]))

  cnv_lyr2_weights = tf.get_variable("W2", shape=[patch_size, patch_size, depth, depth], \
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
  cnv_lyr2_biases = tf.Variable(tf.constant(0.01, shape=[depth]))

  cnv_lyr3_weights = tf.get_variable("W3", shape=[patch_size, patch_size, depth, num_hidden], \
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
  cnv_lyr3_biases = tf.Variable(tf.constant(0.01, shape=[num_hidden]))
  

  length_weights = tf.get_variable("WL", shape=[num_hidden * depth // 4 * depth  // 4 , len_labels], \
                                   initializer=tf.contrib.layers.xavier_initializer())
  length_biases = tf.Variable(tf.constant(0.1, shape=[len_labels]))

  digit1_weights = tf.get_variable("WD1", shape=[num_hidden * depth // 4 * depth  // 4 , num_labels], \
                                   initializer=tf.contrib.layers.xavier_initializer())
  digit1_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))

  digit2_weights = tf.get_variable("WD2", shape=[num_hidden * depth  // 4 * depth  // 4 , num_labels],\
                                   initializer=tf.contrib.layers.xavier_initializer())
  digit2_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))

  digit3_weights = tf.get_variable("WD3", shape=[num_hidden * depth  // 4 * depth  // 4 , num_labels], \
                                   initializer=tf.contrib.layers.xavier_initializer())
  digit3_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))

  digit4_weights = tf.get_variable("WD4", shape=[num_hidden * depth  // 4 * depth  // 4 , num_labels], \
                                   initializer=tf.contrib.layers.xavier_initializer())
  digit4_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))

  digit5_weights = tf.get_variable("WD5", shape=[num_hidden * depth  // 4 * depth  // 4 , num_labels], \
                                   initializer=tf.contrib.layers.xavier_initializer())
  digit5_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))

  def max_pool_2x2(x):
     return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
  
  # Model.
  def model(data, digit_masks_1,digit_masks_2,digit_masks_3,digit_masks_4,digit_masks_5, is_train):
    print("data:",data.get_shape())
    conv = tf.nn.conv2d(data, cnv_lyr1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + cnv_lyr1_biases)
    lrn = tf.nn.local_response_normalization(hidden)
    h_pool1 = max_pool_2x2(lrn)
    print("h_pool1:",h_pool1.get_shape())
    
    conv = tf.nn.conv2d(hidden, cnv_lyr2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + cnv_lyr2_biases)
    lrn = tf.nn.local_response_normalization(hidden)
    h_pool2 = max_pool_2x2(lrn)
    print("h_pool2:",h_pool2.get_shape())

    conv = tf.nn.conv2d(h_pool2, cnv_lyr3_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + cnv_lyr3_biases)
    lrn = tf.nn.local_response_normalization(hidden)
    h_pool3 = max_pool_2x2(lrn)
    print("h_pool3:",h_pool3.get_shape())

    shape = h_pool3.get_shape().as_list()
    print("shape:",shape)
    conv_result_reshape = tf.reshape(h_pool3, [shape[0], shape[1] * shape[2] * shape[3]])
    print("\n conv_result_reshape.get_shape():",conv_result_reshape.get_shape())
    
    print("tf_digit_masks:", digit_masks_1,digit_masks_2,digit_masks_3,digit_masks_4,digit_masks_5)
  
    logit_length = tf.matmul(conv_result_reshape, length_weights) + length_biases
    print("digit1_weights.shape: ",digit1_weights.get_shape())
    print("digit1_biases.shape: ",digit1_biases.get_shape())

    logit_1 = (tf.matmul(conv_result_reshape, digit1_weights) + digit1_biases)*(digit_masks_1 if is_train==True else 1)
    logit_2 = (tf.matmul(conv_result_reshape, digit2_weights) + digit2_biases)*(digit_masks_2 if is_train==True else 1)
    logit_3 = (tf.matmul(conv_result_reshape, digit3_weights) + digit3_biases)*(digit_masks_3 if is_train==True else 1)
    logit_4 = (tf.matmul(conv_result_reshape, digit4_weights) + digit4_biases)*(digit_masks_4 if is_train==True else 1)
    logit_5 = (tf.matmul(conv_result_reshape, digit5_weights) + digit5_biases)*(digit_masks_5 if is_train==True else 1)
    

    return logit_length, logit_1,logit_2,logit_3,logit_4,logit_5
  
  # Training computation.
  logit_len, logit_digit_1,logit_digit_2,logit_digit_3,logit_digit_4,logit_digit_5 \
    = model(tf_train_dataset, tf_digit_masks_1, tf_digit_masks_2, tf_digit_masks_3, tf_digit_masks_4, \
            tf_digit_masks_5, True)

  loss_len     = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_len, tf_train_labels[:,0])
  loss_digit_1 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_1, tf_train_labels[:,1])
  loss_digit_2 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_2, tf_train_labels[:,2])
  loss_digit_3 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_3, tf_train_labels[:,3])
  loss_digit_4 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_4, tf_train_labels[:,4])
  loss_digit_5 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_5, tf_train_labels[:,5])
    
  loss = tf.reduce_mean(loss_len + loss_digit_1 + loss_digit_2 + loss_digit_3 + loss_digit_4 + loss_digit_5)
    
    
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.1
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1000, 0.99, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  print("logit_len:",logit_len.get_shape())
  print("logit_digit_1:",logit_digit_1)
  # Predictions for the training, validation, and test data.
  len_preds = tf.nn.softmax(logit_len)
  print("len_preds:", len_preds.get_shape())
  pads = tf.constant([[0, 0], [4, 0]])
  print("paddings:", pads.get_shape())
  len_preds=tf.pad(len_preds, pads, "CONSTANT")
  print("len_preds:", len_preds.get_shape())
  
  train_prediction = tf.pack([ len_preds,
                               tf.nn.softmax(logit_digit_1),
                               tf.nn.softmax(logit_digit_2),
                               tf.nn.softmax(logit_digit_3),
                               tf.nn.softmax(logit_digit_4),
                               tf.nn.softmax(logit_digit_5)
                             ])
 
 

data: (32, 64, 64, 1)
h_pool1: (32, 32, 32, 64)
h_pool2: (32, 32, 32, 64)
h_pool3: (32, 16, 16, 1024)
shape: [32, 16, 16, 1024]

 conv_result_reshape.get_shape(): (32, 262144)
tf_digit_masks: Tensor("Placeholder_2:0", shape=(32, 10), dtype=float32) Tensor("Placeholder_3:0", shape=(32, 10), dtype=float32) Tensor("Placeholder_4:0", shape=(32, 10), dtype=float32) Tensor("Placeholder_5:0", shape=(32, 10), dtype=float32) Tensor("Placeholder_6:0", shape=(32, 10), dtype=float32)
digit1_weights.shape:  (262144, 10)
digit1_biases.shape:  (10,)
logit_len: (32, 6)
logit_digit_1: Tensor("mul:0", shape=(32, 10), dtype=float32)
len_preds: (32, 6)
paddings: (2, 2)
len_preds: (32, 10)


In [11]:
# Create 5 mask arrays; 1 for each digit.
# Each such Xth array will have all zeroes in Nth a row, if Xh digit is not present in Nth number in batch. 
def explode(batch_labels_arr):
    batch_size_count=batch_labels_arr.shape[0]
    target_col_count=10 # 10 labels for each digit
    total_digits_cols=5
    digit_mask_array = np.ndarray(shape=(total_digits_cols, batch_size_count, target_col_count),dtype=np.int32)
    digit_mask_array[:,:,:]=int(0)
    #print("batch_labels_arr: \n",batch_labels_arr)
    #print(digit_mask_array.shape)
    for row in range(batch_size_count):
        len_val=batch_labels_arr[row,0].astype(int)
        #print("len_val:",len_val)
        for counter in range(len_val):
            digit_mask_array[5-counter-1, row, :] = 1
    #print("digit_mask_array:\n",digit_mask_array)
    return digit_mask_array[0,:,:],digit_mask_array[1,:,:],digit_mask_array[2,:,:],digit_mask_array[3,:,:],digit_mask_array[4,:,:],
    
#print(train_labels[10:20,:])        
#explode(train_labels[10:20,:])

In [12]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print("offset:", offset)
    batch_data = train_dataset[offset:(offset + batch_size), :, :,:]
    #print("batch_data:",batch_data.shape)
    batch_labels = train_labels[offset:(offset + batch_size), :].astype(int)
    digit_1_mask, digit_2_mask,digit_3_mask,digit_4_mask,digit_5_mask = explode(batch_labels)
    #batch_masks=np.array([item == item for item in batch_labels]).astype(int)
    #print("digit_5_mask:",digit_5_mask)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_digit_masks_1 : digit_1_mask ,\
                tf_digit_masks_2 : digit_2_mask, tf_digit_masks_3 : digit_3_mask, tf_digit_masks_4 : digit_4_mask, \
                tf_digit_masks_5 : digit_5_mask}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step > 1 and step % 5000 == 0):
      print("batch_data:",)
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
     # print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
batch_data:
Minibatch loss at step 5000: 29.248043
predictions:
 [[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     2.04795406e-14   2.00054544e-16   1.00000000e+00   1.18811062e-13
     1.14099641e-09   1.20485080e-11]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     3.51005236e-14   1.68919241e-14   1.00000000e+00   3.55518066e-15
     5.23061461e-09   2.06505889e-11]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     6.47420323e-13   4.66090187e-12   1.00000000e+00   2.51213275e-13
     1.77661619e-08   4.08489437e-10]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     2.00027007e-13   9.15654596e-13   1.00000000e+00   1.49614767e-14
     1.45947148e-08   1.09951194e-10]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     6.60100403e-12   1.24892052e-09   9.99999881e-01   1.09969108e-12
     8.35486773e-08   3.89486221e-09]
  [  0.00000000e+00   0

KeyboardInterrupt: 

#### 